## Trabalho 1 (Classificação) - Introdução ao Aprendizado de Máquina (EEL891)
> Nome: Danilo Davi Gomes Fróes
>
> DRE: 124026825

### Análise Exploratória de Dados (EDA - Exploratory Data Analysis)

In [ ]:
import pandas as pd

df_treinamento = pd.read_csv('conjunto_de_treinamento.csv')

print(f'Info: {df_treinamento.info()}')

print(f'Descrição: {df_treinamento.describe()}')

print(f'Targets: {df_treinamento['inadimplente'].value_counts()}')

In [ ]:
df_predicao = pd.read_csv('conjunto_de_teste.csv')

print(f'Info: {df_predicao.info()}')

print(f'Descrição: {df_predicao.describe()}')

### **Tratamento dos Dados**
Aqui é onde vai ser feito o tratamento dos dados após a análise geral. A ideia é deixar apenas dados que possuam valor de predição para o modelo, evitando vieses e análises desnecessárias.

#### **Remoção de colunas**
Algumas colunas foram vistas como problemáticas e serão removidas, elas são:
- `id_solicitante`: Identificador único, não é necessário para o modelo.
- `grau_instrucao`: Veio preenchida apenas com zeros.
- `possui_telefone_celular`: Veio preenchida apenas com `'N'`.
- `qtde_contas_bancarias_especiais`: Idêntica a outra coluna, `'qtde_contas_bancarias'`.

#### **Substituição de valores nulos ocultos**
Foi observado pelo dicionário de dados, que existem valores nulos que estão ocultos no dataframe, ou seja, não seriam enxergues como nulos no pré-processamento. Valores como `"XX"` e espaços em branco `" "` serão substituídos por valores nulos NaN

In [ ]:
import numpy as np

# Remover colunas que não são necessárias para o modelo
colunas_a_remover = [
    'id_solicitante',
    'grau_instrucao',
    'possui_telefone_celular',
    'qtde_contas_bancarias_especiais'
]
df_treinamento = df_treinamento.drop(columns=colunas_a_remover, axis=1)

# Substituir valores inválidos por NaN
df_treinamento = df_treinamento.replace([' ', 'N/A', '', '?', 'XX', 'NULL'], np.nan)

# Engenharia de Features
df_treinamento['renda_total'] = df_treinamento['renda_mensal_regular'] + df_treinamento['renda_extra']
df_treinamento['patrimonio_por_renda'] = df_treinamento['valor_patrimonio_pessoal'] / (df_treinamento['renda_total'] + 1)
df_treinamento['renda_total_log'] = np.log1p(df_treinamento['renda_total'])
df_treinamento['nasceu_onde_reside'] = (df_treinamento['estado_onde_nasceu'] == df_treinamento['estado_onde_reside']).astype(int)
df_treinamento['reside_onde_trabalha'] = (df_treinamento['estado_onde_reside'] == df_treinamento['estado_onde_trabalha']).astype(int)
colunas_cartoes = ['possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes']
df_treinamento['qtde_total_de_cartoes'] = df_treinamento[colunas_cartoes].sum(axis=1)

# Tratamento de valores extremos
df_treinamento.loc[df_treinamento['qtde_dependentes'] > 10, 'qtde_dependentes'] = 10
df_treinamento.loc[df_treinamento['idade'] < 17, 'idade'] = df_treinamento['idade'].median()
# df_treinamento.loc[df_treinamento['meses_no_trabalho'] < 1, 'meses_no_trabalho'] = df_treinamento['meses_no_trabalho'].median()

colunas_redundantes = [
    'renda_mensal_regular', 
    'renda_extra',
    'renda_total',
    'valor_patrimonio_pessoal'
]

df_treinamento = df_treinamento.drop(columns=colunas_redundantes, errors='ignore')

# Tratamento de valores nulos
cols_na_subs = ['profissao_companheiro', 'grau_instrucao_companheiro', 'codigo_area_telefone_residencial', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao']
for col in cols_na_subs:
    df_treinamento[col] = df_treinamento[col].fillna(-1)

cols_cat_subs = ['estado_onde_trabalha', 'estado_onde_nasceu', 'tipo_residencia'] 
for col in cols_cat_subs:
    moda = df_treinamento[col].mode()[0]
    df_treinamento[col] = df_treinamento[col].fillna(moda)

cols_num_subs = ['meses_na_residencia'] 
for col in cols_num_subs:
    mediana = df_treinamento[col].median()
    df_treinamento[col] = df_treinamento[col].fillna(mediana)

df_treinamento['sexo'] = df_treinamento['sexo'].fillna('N')

df_treinamento.info()


In [ ]:
# Remover colunas que não são necessárias para o modelo
colunas_a_remover = [
    'id_solicitante',
    'grau_instrucao',
    'possui_telefone_celular',
    'qtde_contas_bancarias_especiais'
]
df_predicao = df_predicao.drop(columns=colunas_a_remover, axis=1)

# Substituir valores inválidos por NaN
df_predicao = df_predicao.replace([' ', 'N/A', '', '?', 'XX', 'NULL'], np.nan)

# Engenharia de Features
df_predicao['renda_total'] = df_predicao['renda_mensal_regular'] + df_predicao['renda_extra']
df_predicao['patrimonio_por_renda'] = df_predicao['valor_patrimonio_pessoal'] / (df_predicao['renda_total'] + 1)
df_predicao['renda_total_log'] = np.log1p(df_predicao['renda_total'])
df_predicao['nasceu_onde_reside'] = (df_predicao['estado_onde_nasceu'] == df_predicao['estado_onde_reside']).astype(int)
df_predicao['reside_onde_trabalha'] = (df_predicao['estado_onde_reside'] == df_predicao['estado_onde_trabalha']).astype(int)
colunas_cartoes = ['possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes']
df_predicao['qtde_total_de_cartoes'] = df_predicao[colunas_cartoes].sum(axis=1)

# Tratamento de valores extremos
df_predicao.loc[df_predicao['qtde_dependentes'] > 10, 'qtde_dependentes'] = 10
df_predicao.loc[df_predicao['idade'] < 17, 'idade'] = df_predicao['idade'].median()
# df_predicao.loc[df_predicao['meses_no_trabalho'] < 1, 'meses_no_trabalho'] = df_predicao['meses_no_trabalho'].median()

colunas_redundantes = [
    'renda_mensal_regular', 
    'renda_extra',
    'renda_total',
    'valor_patrimonio_pessoal'
]

df_predicao = df_predicao.drop(columns=colunas_redundantes, errors='ignore')

# Tratamento de valores nulos
cols_na_subs = ['profissao_companheiro', 'grau_instrucao_companheiro', 'codigo_area_telefone_residencial', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao']
for col in cols_na_subs:
    df_predicao[col] = df_predicao[col].fillna(-1)

cols_cat_subs = ['estado_onde_trabalha', 'estado_onde_nasceu', 'tipo_residencia'] 
for col in cols_cat_subs:
    moda = df_predicao[col].mode()[0]
    df_predicao[col] = df_predicao[col].fillna(moda)

cols_num_subs = ['meses_na_residencia'] 
for col in cols_num_subs:
    mediana = df_predicao[col].median()
    df_predicao[col] = df_predicao[col].fillna(mediana)

df_predicao['sexo'] = df_predicao['sexo'].fillna('N')

df_predicao.info()

In [ ]:
num_cols = [
    'idade',
    'qtde_dependentes',
    'meses_na_residencia',
    'renda_mensal_regular',
    'renda_extra',
    'qtde_contas_bancarias',
    'qtde_contas_bancarias_especiais',
    'valor_patrimonio_pessoal',
    'meses_no_trabalho'
]

cat_cols = [
    'produto_solicitado',
    'dia_vencimento',
    'forma_envio_solicitacao',
    'tipo_endereco',
    'sexo',
    'estado_civil',
    'grau_instrucao',
    'nacionalidade',
    'estado_onde_nasceu',
    'estado_onde_reside',
    'possui_telefone_residencial',
    'codigo_area_telefone_residencial',
    'tipo_residencia',
    'possui_telefone_celular',
    'possui_email',
    'possui_cartao_visa',
    'possui_cartao_mastercard',
    'possui_cartao_diners',
    'possui_cartao_amex',
    'possui_outros_cartoes',
    'possui_carro',
    'vinculo_formal_com_empresa',
    'estado_onde_trabalha',
    'possui_telefone_trabalho',
    'codigo_area_telefone_trabalho',
    'profissao',
    'ocupacao',
    'profissao_companheiro',
    'grau_instrucao_companheiro'
]

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
# from xgboost import XGBClassifier

scalers = [
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
    MaxAbsScaler
]

encoders = [
    OneHotEncoder,
    OrdinalEncoder,
]

imputers = [
    SimpleImputer(strategy='mean')
]

models = {
    # 'RandomForest': {
    #     'model': RandomForestClassifier(random_state=2),
    #     'hyperparameters': {
    #         'n_estimators': [100, 200],
    #         'max_depth': [5, 10, None],
    #         'min_samples_split': [2, 5, 10]
    #     },
    #     'selection_method': 'grid',
    #     'scoring': 'accuracy'
    # },

    # 'SVM': {
    #     'model': SVC(random_state=2),
    #     'hyperparameters': {
    #         'C': [0.1, 1, 10],
    #         'kernel': ['rbf', 'linear'],
    #         'gamma': ['scale', 'auto']
    #     },
    #     'selection_method': 'grid',
    #     'scoring': 'accuracy',
    #     'cv': 5
    # },

    'LogisticRegression': {
        'model': LogisticRegression(random_state=2),
        'hyperparameters': {
            'C': [0.1, 1, 10],
            'solver': ['liblinear', 'saga']
        }
    },

    # 'GradientBoosting': {
    #     'model': GradientBoostingClassifier(random_state=2),
    #     'hyperparameters': {
    #         'n_estimators': [100, 200],
    #         'learning_rate': [0.01, 0.1, 0.2],
    #         'max_depth': [3, 5, 7]
    #     }
    # },

    # 'XGBoost': {
    #     'model': XGBClassifier(random_state=2),
    #     'hyperparameters': {
    #         'n_estimators': [100, 200],
    #         'learning_rate': [0.01, 0.1, 0.3],
    #         'max_depth': [3, 6, 10]
    #     }
    # },

    # 'AdaBoost': {
    #     'model': AdaBoostClassifier(random_state=2),
    #     'hyperparameters': {
    #         'n_estimators': [50, 100, 200],
    #         'learning_rate': [0.01, 0.1, 1]
    #     },
    #     'selection_method': 'grid',
    #     'scoring': 'accuracy'
    # },

    # 'ExtraTrees': {
    #     'model': ExtraTreesClassifier(random_state=2),
    #     'hyperparameters': {
    #         'n_estimators': [100, 200],
    #         'max_depth': [5, 10, None],
    #         'min_samples_split': [2, 5, 10]
    #     },
    #     'selection_method': 'grid',
    #     'scoring': 'accuracy'
    # },

    # 'KNN': {
    #     'model': KNeighborsClassifier(),
    #     'hyperparameters': {
    #         'n_neighbors': [3, 5, 7],
    #         'weights': ['uniform', 'distance'],
    #         'metric': ['euclidean', 'manhattan']
    #     },
    #     'selection_method': 'grid',
    #     'scoring': 'accuracy'
    # },

    # 'DecisionTree': {
    #     'model': DecisionTreeClassifier(random_state=2),
    #     'hyperparameters': {
    #         'max_depth': [5, 10, None],
    #         'min_samples_split': [2, 5, 10],
    #         'criterion': ['gini', 'entropy']
    #     },
    #     'selection_method': 'grid',
    #     'scoring': 'accuracy'
    # }
}

